In [1]:
# IO packages
from aicsimageio import AICSImage
import os
import imageio

# calculation packages
import numpy as np

# segmentation packages
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from aicssegmentation.core.seg_dot import dot_3d_wrapper
from skimage.morphology import dilation, ball, remove_small_objects

# watershed packages
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy.ndimage import distance_transform_edt
from skimage.measure import label



In [2]:
def segment_RNA(image_name, filepath, s3_param, output_path):
    """ Takes a filepath that is a single channel tif as an input
    Segments the image - optimized for RNA
    Returns a segmented image
    """
        
    print('Segmenting RNA for ' + filepath)

    
    reader = AICSImage(filepath) 
    IMG = reader.data

    #####################
    structure_channel = 0
    #####################

    structure_img0 = IMG[0,structure_channel,:,:,:]
        
    ################################
    # First, calculate the best intensity normalization parameters
    
    minimum_value = np.amin(structure_img0)
    mean_value = np.mean(structure_img0)
    percentile_99 = np.percentile(structure_img0, 99.99)
    std_array = np.std(structure_img0)

    a = round((mean_value - minimum_value) / std_array, 1)
    b = round((percentile_99 - mean_value) / std_array, 1)
    
    ################################
    ## PARAMETER ##
    intensity_scaling_param = [a, b]
    gaussian_smoothing_sigma = 0.5
    ################################

    # intensity normalization
    structure_img = intensity_normalization(structure_img0, scaling_param=intensity_scaling_param)

    # smoothing with gaussian filter
    structure_img_smooth = image_smoothing_gaussian_slice_by_slice(structure_img, sigma=gaussian_smoothing_sigma)
    
    ################################
    ## Detect spots w/ 3D dot wrapper ##
    ## Parameters are optimized to stage and RNA type ##
    bw = dot_3d_wrapper(structure_img_smooth, s3_param)

    #################################
    # Watershed to resolve close together particles
   
    minArea = 20
    local_maxi = peak_local_max(structure_img0,labels=label(bw), min_distance=5, indices=False)
    distance = distance_transform_edt(bw)
    im_watershed = watershed(-distance, label(dilation(local_maxi, selem=ball(1))), mask=bw, watershed_line=True)
  
    ################################
    ## PARAMETERS for removing small objects ##
    minArea = 20
    ################################

    final_seg = remove_small_objects(im_watershed>0, min_size=minArea, connectivity=1, in_place=False)
    
    ################################
    ## SAVE TO DISK
    ################################
    
    output_seg = final_seg>0
    out=output_seg.astype(np.uint8)
    out[out>0]=255
    imageio.volwrite(output_path + image_name, out)

        
    return None

In [3]:

s3_params_dict = {'cen NC13': [[1, 0.005], [10, 0.1]],
                 'cen NC10': [[1, 0.03]],
                 'cg14438': [[1, 0.05]],
                 'cyc B': [[1, 0.075]],
                 'gapdh': [[1, 0.075]],
                  'pins': [[1, 0.05]],
                  'plp': [[1, 0.05]]
                 }


In [4]:
rna_dirs_to_process = ['gapdh']

filepath_to_dirs = '/Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/'

output_path = '/Users/pearlryder/data/resubmission-analysis-2/data/rna/segmentations/'

for rna_dir in rna_dirs_to_process:
    
    dir_path = os.path.join(filepath_to_dirs, rna_dir)
    
    image_ls = os.listdir(dir_path)
    
    for image_name in image_ls:
        if not image_name[0] == '.':
            image_path = os.path.join(dir_path, image_name)

            s3_param = s3_params_dict[rna_dir]

            segment_RNA(image_name, image_path, s3_param, output_path)
        

    
    


Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC13_metaphase_slide254_img_007.tif
intensity normalization: normalize into [mean - 3.6 x std, mean + 15.1 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC13_metaphase_slide254_img_013.tif
intensity normalization: normalize into [mean - 3.8 x std, mean + 15.2 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC13_interphase_Slide_261_Img_015.tif
intensity normalization: normalize into [mean - 3.6 x std, mean + 14.1 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC10_metaphase_slide265_img001.tif
intensity normalization: normalize into [mean - 4.1 x std, mean + 8.4 x std] 
intensity normalization completes
Segmenting RNA f

intensity normalization: normalize into [mean - 2.8 x std, mean + 10.7 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC10_interphase_slide267_img006.tif
intensity normalization: normalize into [mean - 3.0 x std, mean + 10.2 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC11_interphase_Slide038_Emb29_Img1.tif
intensity normalization: normalize into [mean - 3.6 x std, mean + 9.1 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC10_metaphase_Slide065_Emb015.tif
intensity normalization: normalize into [mean - 3.9 x std, mean + 12.9 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC10_interphase_slide267_img002.tif
intensity norm

intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC13_interphase_slide254_img_010.tif
intensity normalization: normalize into [mean - 3.4 x std, mean + 13.4 x std] 
intensity normalization completes
Segmenting RNA for /Users/pearlryder/data/resubmission-analysis-2/data/rna/raw-data-by-rna/gapdh/NC10_metaphase_Slide038_Emb43_Img1.tif
intensity normalization: normalize into [mean - 4.6 x std, mean + 10.1 x std] 
intensity normalization completes
